In [1]:
import torch

from quant_ops.quant_bmm import BMM_S8T_S8N_S8O

a_tensor: torch.Tensor = torch.randn(1, 1, 4, 5)
b_tensor: torch.Tensor = torch.randn(1, 1, 5, 4)

c_tensor = torch.matmul(a_tensor, b_tensor)
print(c_tensor)

tensor([[[[ 1.4984,  1.0325,  0.8960, -2.2726],
          [ 3.7410,  0.8018, -1.8444,  0.6214],
          [-2.6356, -0.1725,  2.4648, -1.0150],
          [-1.1821,  1.5867, -0.5682, -0.1705]]]])


In [2]:
from supports.quant_base_method import quant_per_tensor_asymmetric_int8

q_a_tensor, a_scales, a_zp = quant_per_tensor_asymmetric_int8(a_tensor)
q_b_tensor, b_scales, b_zp = quant_per_tensor_asymmetric_int8(b_tensor)
q_c_tensor, c_scales, c_zp = quant_per_tensor_asymmetric_int8(c_tensor)

In [3]:
ops: BMM_S8T_S8N_S8O = BMM_S8T_S8N_S8O.from_a_and_b(
    a_zp=a_zp,
    b_zp=b_zp,
    o_zp=c_zp,
    a_scale=a_scales,
    b_scale=b_scales,
    o_scale=c_scales,
)

In [4]:
q_c = ops(q_a_tensor, q_b_tensor)

In [5]:
print(q_c)

tensor([[[[  35,   18,   11, -113],
          [ 126,    9,  -96,    1],
          [-128,  -29,   75,  -63],
          [ -71,   41,  -46,  -30]]]], dtype=torch.int8)


In [6]:
from supports.quant_base_method import dequantize_asymmetric_quantize

f32_o = dequantize_asymmetric_quantize(q_c, c_scales, c_zp)
print(f32_o)

tensor([[[[ 1.4504,  1.0253,  0.8502, -2.2506],
          [ 3.7259,  0.8002, -1.8254,  0.6001],
          [-2.6256, -0.1500,  2.4506, -1.0002],
          [-1.2003,  1.6004, -0.5751, -0.1750]]]])
